Import necessary libraries.

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torch.optim as optim
import torchvision.transforms.functional as TF
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import csv
import cv2
from tqdm import tqdm
import os
import json
from os.path import exists
import matplotlib.pyplot as plt
from skimage import transform

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


**Load your path.** 

Set the path in your drive,the folder need to contain an **empty subfolder "metrics"**, one **subfolder "testset"** containing the test set and similarly one **subfolder "trainset"** for the train set.

In [ ]:
import sys
## Change it with the place that you have placed the data with the format described above ##
from google.colab import drive
drive.mount('/content/drive')
sys.path.append('drive/My Drive/shared')
path='drive/My Drive/shared/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Usefull functions/utils:

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


def plot_metrics(name: str):
    data = []
    headers = None
    with open(name, 'r') as f:
        reader = csv.reader(f, delimiter=',')
        for row in reader:
            data.append(row)
    train_metrics = []
    test_metrics = []
    for idx,row in enumerate(data):
        if idx == 0:
            headers = row
        elif (idx % 2) == 1:
            train_metrics.append(row)
        elif (idx % 2) == 0:
            test_metrics.append(row)

    epochs = []
    for i in range(len(train_metrics)):
        epochs.append(i)
    recall_train = []
    recall_test = []
    dice_train = []
    dice_test = []
    for i in train_metrics:
        recall_train.append( float("{:.2f}".format(float(i[0]))) )
    for i in test_metrics:
        recall_test.append(float("{:.2f}".format(float(i[0]))))
    for i in train_metrics:
        dice_train.append( float("{:.2f}".format(float(i[1]))) )
    for i in test_metrics:
        dice_test.append(float("{:.2f}".format(float(i[1]))))

    fig = plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot( epochs , recall_train, label='train_loss')
    plt.plot( epochs, recall_test, label='valid_loss')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot( epochs, dice_train, label='train_accs')
    plt.plot(epochs, dice_test, label='valid_accs')
    plt.legend()
    plt.savefig(path+'/metrics/metrics_baseline.png')


def save_metrics(metrics,name: str):
    data = []
    file_exists = exists(name)
    header = ['Recal', 'Dice', 'Tversky', 'Cross Entropy', 'Total', 'IoU']
    for loader in metrics:
        for i in range(len(loader)):
            if i ==0:
                loader[i] =  float ( "{:.2f}".format( float( loader[i].cpu().numpy()) ) )
            else:
                loader[i] = float("{:.4f}".format(float(loader[i].cpu().numpy())))
        data.append(loader)
    with open(name, 'a',newline='') as f:
        writer = csv.writer(f)
        if not file_exists:
            writer.writerow(header)
        for i in data:
            writer.writerow(i)
    plot_metrics(name)


def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)


def load_checkpoint(checkpoint, model):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])


def get_loaders(train_dir , test_dir, batch_size, train_transform, test_transform, num_workers=4, pin_memory=True):
    train_ds = create_dataset(image_dir=train_dir, train=True, transform = train_transform)

    train_loader = DataLoader(train_ds, batch_size=batch_size,num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=True)

    test_ds = create_dataset(
        image_dir=test_dir, train=False, transform = test_transform
    )

    test_loader = DataLoader(
        test_ds,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=False,
    )

    return train_loader, test_loader


def check_accuracy( train_loader ,test_loader, model, device="cuda"):
    TP_test = 0.0
    FN_test = 0.0
    total_test = 0.0
    union_test = 0.0
    TP_test_modified = 0.0
    FN_test_modified = 0.0
    dice_score_test = 0.0
    cross_loss_test = 0.0
    total_loss_test = 0.0
    Tversky_test = 0.0

    TP_train = 0.0
    FN_train = 0.0
    total_train = 0.0
    TP_train_modified = 0.0
    FN_train_modified = 0.0
    dice_score_train = 0.0
    cross_loss_train = 0.0
    total_loss_train = 0.0
    Tversky_train = 0.0

    model.eval()

    #Set loss function
    total = Total_loss()

    with torch.no_grad():
        for all_data in test_loader:
            x = all_data[:, 0, :, :]
            y = all_data[:, 1:10, :, :]
            x = x.float().unsqueeze(1).to(device=DEVICE)
            y = y.float().to(device=DEVICE)

            preds = model(x)
            loss = total(preds, y)
            total_loss_test += loss
            cross_loss_test += total.ce
            Tversky_test += total.tversky

            preds = torch.softmax(preds, 1)
            preds = (torch.zeros(preds.shape).scatter(1, preds.argmax(1).unsqueeze(1).cpu(), 1.0)).to(device=DEVICE)

            TP_test_modified += (preds[:, 1:9, :, :] * y[:, 1:9, :, :]).sum()
            FN_test_modified += (y[:, 1:9, :, :] * (1 - preds[:, 1:9, :, :])).sum()
            TP_test += (preds * y).sum()
            FN_test += (y * (1 - preds)).sum()
            total_test += (preds + y).sum()

            dice_score_test += (2 * (preds * y).sum()) / (
                (preds + y).sum() + 1e-8
            )
    union_test += total_test - TP_test
    print(
        f"Testing set:...Recall: {TP_test/(TP_test+FN_test)*100:.2f} ,...Car Recall: {TP_test_modified/(TP_test_modified+FN_test_modified)*100} ,...Dice score: {1-dice_score_test/len(test_loader)} "
        f",...Tversky loss: {Tversky_test/len(test_loader)},...Cross_entropy: {cross_loss_test/len(test_loader)}"
    )

    with torch.no_grad():
        for all_data in train_loader:
            x = all_data[:, 0, :, :]
            y = all_data[:, 1:10, :, :]
            x = x.float().unsqueeze(1).to(device=DEVICE)
            y = y.float().to(device=DEVICE)

            preds = model(x)
            loss = total(preds, y)
            total_loss_train += loss
            cross_loss_train += total.ce
            Tversky_train += total.tversky

            preds = torch.softmax(preds, 1)
            preds = (torch.zeros(preds.shape).scatter(1, preds.argmax(1).unsqueeze(1).cpu(), 1.0)).to(device=DEVICE)

            TP_train_modified += (preds[:, 1:9, :, :] * y[:, 1:9, :, :]).sum()
            FN_train_modified += (y[:, 1:9, :, :] * (1 - preds[:, 1:9, :, :])).sum()
            TP_train += (preds * y).sum()
            FN_train += (y * (1 - preds)).sum()
            total_train += (preds + y).sum()


            dice_score_train += (2 * (preds * y).sum()) / (
                    (preds + y).sum() + 1e-8
            )
        union_train = total_train - TP_train
        print(
            f"Training set:...Recall: {TP_train / (TP_train + FN_train) * 100:.2f} ,...Car Recall: {TP_train_modified/(TP_train_modified+FN_train_modified)*100} ,...Dice score: {1 - dice_score_train / len(train_loader)} "
            f",...Tversky loss: {Tversky_train / len(train_loader)},...Cross_entropy: {cross_loss_train / len(train_loader)}"
        )

    model.train()

    #Return [ [train_metris] , [test_metrics] ] = [ [Recall,Dice,Tversky(used),Cross-Entropy(used),Total(used),IoU, Recall_modified] , [Recall,...,Recall_modified] ]
    return [
                [(TP_train/(TP_train+FN_train))*100, 1 - (dice_score_train/len(train_loader)),
                 (Tversky_train/len(train_loader)),
                 (cross_loss_train/len(train_loader)), (total_loss_train/len(train_loader)),
                 (TP_train+1e-8)/(union_train+1e-8),
                 (TP_train_modified/(TP_train_modified+FN_train_modified))*100],
                [(TP_test / (TP_test + FN_test)) * 100, 1 - (dice_score_test / len(test_loader)),
                 (Tversky_test / len(test_loader)),
                 (cross_loss_test / len(test_loader)), (total_loss_test / len(test_loader)),
                 (TP_test + 1e-8) / (union_test + 1e-8),
                 (TP_test_modified/(TP_test_modified+FN_test_modified))*100]
            ]


def check_top_five(dir, name, metric, model_snapshot):
    dictionary = {
            "checkpoint_0": float('inf'),
            "checkpoint_1": float('inf'),
            "checkpoint_2": float('inf'),
            "checkpoint_3": float('inf'),
            "checkpoint_4": float('inf')
    }
    if not exists(dir+name):
        json_object = json.dumps(dictionary)
        with open(dir+name, "w") as outfile:
            outfile.write(json_object)
        outfile.close()

    f = open(dir+name, 'r')
    data = json.load(f)
    data = dict(sorted(data.items(), key=lambda item: item[1], reverse=True))
    checkpoint = next(iter(data))
    print(f"{data[checkpoint]} > {metric}????")
    if data[checkpoint] > metric:
        data[checkpoint] = metric
        save_checkpoint(model_snapshot, filename=dir+checkpoint+".pth.tar")
    else:
        print("Checkpoint is not in top 5 :'(")
    f.close()

    with open(dir + name, "w") as file:
        json_object = json.dumps(data)
        file.write(json_object)
    file.close()


def save_predictions_as_imgs(
    loader, model, folder="saved_images/", device="cuda"
):
    model.eval()
    for idx, all_data in enumerate(loader):
        x = all_data[:, 0, :, :]
        y = all_data[:, 1:10, :, :]
        x = x.float().unsqueeze(1).to(device=DEVICE)
     #   y = y.float().unsqueeze(1)


        with torch.no_grad():
            preds = torch.softmax(model(x),1)
            preds = preds.cpu()
            #preds = torch.zeros(preds.shape).scatter(1, preds.argmax(1).unsqueeze(1), 1.0)
            #preds = (preds = max_preds).float()
            preds = preds.cuda()

        for itr in range(9):
            torchvision.utils.save_image(
                torch.zeros(preds.shape).scatter(1, preds.argmax(1).unsqueeze(1).cpu(), 1.0)[:, itr, :, :].unsqueeze(1).cuda(), f"{folder}/pred_{idx}_itr_{itr}.png"
            )
            torchvision.utils.save_image(preds[:, itr, :, :].unsqueeze(1), f"{folder}{idx}_grey_{itr}.png")

    model.train()

Create data loaders:

In [ ]:
class create_dataset(Dataset):
    def __init__(self, image_dir, train, transform=None):

        self.image_dir = image_dir
        self.original_images = os.listdir(image_dir)

        self.transform = transform


    def __len__(self):
        return len(self.original_images)

    def __getitem__(self, index):
        image_path = os.path.join(self.image_dir, self.original_images[index])
        image = np.load(image_path)



        if self.transform is not None:
            image = self.transform(image)

        return image

Set transformations:

In [ ]:
class Normalize(object):
    """Normalize image between 0-1
        """
    def __call__(self,sample):
        sample[0, :, :] = 255 * (sample[0, :, :] - np.min(sample[0, :, :])) / np.ptp(sample[0, :, :])
        sample[1, :, :] = 255 * (sample[1, :, :] - np.min(sample[1, :, :])) / np.ptp(sample[1, :, :])
        sample[2, :, :] = 255 * (sample[2, :, :] - np.min(sample[2, :, :])) / np.ptp(sample[2, :, :])

        #sample[0, :, :] = (sample[0, :, :] + 0.485 / 0.229 - 0.485) / 0.229
        #sample[1, :, :] = (sample[1, :, :] + 0.456 / 0.224 - 0.456) / 0.224
        #sample[2, :, :] = (sample[2, :, :] + 0.406 / 0.225 - 0.406) / 0.225

        return sample

class Rescale(object):
    """Rescale the image in a sample to a given size.
    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, output_size):
        self.output_size = output_size

    def __call__(self, sample):
        sample = np.transpose(sample, (1, 2, 0))
        image = sample[:, :, :3]
        masks = sample[:, :, 3:]

        h, w = image.shape[:2]

        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h/w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w/h
        else:
            new_h, new_w = self.output_size

        new_h, new_w = int(new_h), int(new_w)

        image = transform.resize(image, (new_h, new_w))

        masks = transform.resize(masks, (new_h, new_w))
        rescaled_sample = np.concatenate((image, masks), axis=2)

        return rescaled_sample

class Grayscale(object):
    """Convert RBG image in sample to Grayscale."""
    def __call__(self, sample):

        image = sample[:, :, :3]
        masks = sample[:, :, 3:]

        h, w = image.shape[:2]

        gray_image = np.zeros((h,w,11))
        gray_image[:,:, 1:] = masks
        gray_image[:, :,0] =np.dot(sample[:, :, :3], [0.299, 0.587, 0.144])

        return gray_image

class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        sample = sample.transpose((2, 0, 1))
        return torch.from_numpy(sample)

class randomHueSaturationValue(object):

    def __init__(self, hue_shift_limit=(-180, 180),
                             sat_shift_limit=(-255, 255),
                             val_shift_limit=(-255, 255), u=0.5):

        self.hue_shift_limit = hue_shift_limit
        self.sat_shift_limit = sat_shift_limit
        self.val_shift_limit = val_shift_limit
        self.u = u

    def __call__(self,samples):

        image = samples[:,:,:3]

        image = np.array(image, np.uint8)
        image = image[:, :, ::-1].copy()

        if np.random.random() < self.u:
            image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            h, s, v = cv2.split(image)
            hue_shift = np.random.uniform(self.hue_shift_limit[0], self.hue_shift_limit[1])
            h = cv2.add(h, hue_shift)
            sat_shift = np.random.uniform(self.sat_shift_limit[0], self.sat_shift_limit[1])
            s = cv2.add(s, sat_shift)
            val_shift = np.random.uniform(self.val_shift_limit[0], self.val_shift_limit[1])
            v = cv2.add(v, val_shift)
            image = cv2.merge((h, s, v))
            image = cv2.cvtColor(image, cv2.COLOR_HSV2RGB)

        image = image[:, :, ::-1].copy()

        samples[:,:,:3] = image

        return samples

class randomShiftScaleRotate(object):

    def __init__(self,shift_limit=(-0.0625, 0.0625),
                           scale_limit=(-0.1, 0.1),
                           rotate_limit=(-45, 45), aspect_limit=(0, 0),
                           borderMode=cv2.BORDER_CONSTANT, u=0.5):

        self.shift_limit = shift_limit
        self.scale_limit = scale_limit
        self.rotate_limit = rotate_limit
        self.aspect_limit = aspect_limit
        self.borderMode = borderMode
        self.u = u


    def __call__(self,samples):


        image = np.array(samples[:,:,:3], np.uint8)
        mask = np.array(samples[:,:,3:], np.uint8)
        image = image[:, :, ::-1].copy()
        mask = mask[:, :, ::-1].copy()

        if np.random.random() < self.u:
            height, width, channel = image.shape

            angle = np.random.uniform(self.rotate_limit[0], self.rotate_limit[1])  # degree
            scale = np.random.uniform(1 + self.scale_limit[0], 1 + self.scale_limit[1])
            aspect = np.random.uniform(1 + self.aspect_limit[0], 1 + self.aspect_limit[1])
            sx = scale * aspect / (aspect ** 0.5)
            sy = scale / (aspect ** 0.5)
            dx = round(np.random.uniform(self.shift_limit[0], self.shift_limit[1]) * width)
            dy = round(np.random.uniform(self.shift_limit[0], self.shift_limit[1]) * height)

            cc = np.math.cos(angle / 180 * np.math.pi) * sx
            ss = np.math.sin(angle / 180 * np.math.pi) * sy
            rotate_matrix = np.array([[cc, -ss], [ss, cc]])

            box0 = np.array([[0, 0], [width, 0], [width, height], [0, height], ])
            box1 = box0 - np.array([width / 2, height / 2])
            box1 = np.dot(box1, rotate_matrix.T) + np.array([width / 2 + dx, height / 2 + dy])

            box0 = box0.astype(np.float32)
            box1 = box1.astype(np.float32)
            mat = cv2.getPerspectiveTransform(box0, box1)
            image = cv2.warpPerspective(image, mat, (width, height), flags=cv2.INTER_LINEAR, borderMode=self.borderMode,
                                        borderValue=(0, 0, 0,))
            mask = cv2.warpPerspective(mask, mat, (width, height), flags=cv2.INTER_LINEAR, borderMode=self.borderMode,
                                       borderValue=(0, 0, 0,))
            if len(mask.shape) == 2:
                mask = np.expand_dims(mask, axis=2)
            if len(image.shape) == 2:
                image = np.expand_dims(image, axis=2)


        samples[:, :, :3]=image
        samples[:, :, 3:] = mask

        return samples

class randomHorizontalFlip(object):

    def __init__(self, u=0.5):
        self.u = u

    def __call__(self,samples):
        image = np.array(samples[:,:,:3], np.uint8)
        mask = np.array(samples[:,:,3:], np.uint8)
        image = image[:, :, ::-1].copy()
        mask = mask[:, :, ::-1].copy()

        if np.random.random() < self.u:
            image = cv2.flip(image, 1)
            mask = cv2.flip(mask, 1)
            if len(image.shape) == 2:
                image = np.expand_dims(image, axis=2)


        samples[:, :, :3] = image
        samples[:, :, 3:] = mask

        return samples

class randomZoom(object):
    def __init__(self, zoom_limit=0.25, u=0.5):

        self.zoom_limit = zoom_limit
        self.u = u

    def __call__(self,samples):
        image = np.array(samples[:,:,:3], np.uint8)
        mask = np.array(samples[:,:,3:], np.uint8)
        image = image[:, :, ::-1].copy()
        mask = mask[:, :, ::-1].copy()

        if np.random.random() < self.u:
            value = np.random.uniform(self.zoom_limit, 1)
            h, w = image.shape[:2]
            h_taken = int(value * h)
            w_taken = int(value * w)
            h_start = np.random.randint(0, h - h_taken)
            w_start = np.random.randint(0, w - w_taken)
            image = image[h_start:h_start + h_taken, w_start:w_start + w_taken, :]
            mask = mask[h_start:h_start + h_taken, w_start:w_start + w_taken, :]
            image = cv2.resize(image, (h, w), cv2.INTER_CUBIC)
            mask = cv2.resize(mask, (h, w), cv2.INTER_CUBIC)
            if len(image.shape) == 2:
                image = np.expand_dims(image, axis=2)

        image = image[:, :, ::-1].copy()
        mask = mask[:, :, ::-1].copy()

        samples[:, :, :3] = image
        samples[:, :, 3:] = mask

        return samples

Import model:

In [ ]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.conv(x)

class UNET(nn.Module):
    def __init__(
            self, in_channels=1, out_channels=1, features=[64, 128, 256, 512],
    ):
        super(UNET, self).__init__()
        self.ups = nn.ModuleList()
        self.downs = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Down part of UNET
        for feature in features:
            self.downs.append(DoubleConv(in_channels, feature))
            in_channels = feature

        # Up part of UNET
        for feature in reversed(features):
            self.ups.append(
                nn.ConvTranspose2d(
                    feature*2, feature, kernel_size=2, stride=2,
                )
            )
            self.ups.append(DoubleConv(feature*2, feature))

        self.bottleneck = DoubleConv(features[-1], features[-1]*2)
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)

    def forward(self, x):
        skip_connections = []

        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)

        x = self.bottleneck(x)
        skip_connections = skip_connections[::-1]

        for idx in range(0, len(self.ups), 2):
            x = self.ups[idx](x)
            skip_connection = skip_connections[idx//2]

            if x.shape != skip_connection.shape:
                x = TF.resize(x, size=skip_connection.shape[2:])

            concat_skip = torch.cat((skip_connection, x), dim=1)
            x = self.ups[idx+1](concat_skip)

        return self.final_conv(x)

def test():
    x = torch.randn((3, 1, 161, 161))
    model = UNET(in_channels=1, out_channels=9)
    preds = model(x)
    print(preds.shape)

if __name__ == "__main__":
    test()

torch.Size([3, 9, 161, 161])


Create costum loss function:

In [ ]:
import torch.nn as nn
import torch
import torch.nn.functional as F
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

class Total_loss(nn.Module):
    def __init__(self, weight=None, size_average=True, alpha=0.3, beta=0.7, sigma=0.4 , theta=0.6  ):
        super(Total_loss, self).__init__()
        self.tversky = 0.0
        self.ce = 0.0
        self.alpha = alpha
        self.beta = beta
        self.sigma = sigma
        self.theta = theta

    def forward(self, inputs, targets, smooth=1e-4):
        #class_weight = torch.tensor([ 0.04, 0.12 , 0.12 , 0.12 , 0.12 , 0.12 , 0.12, 0.12, 0.12])
        class_weight = torch.tensor([ 0.5, 1 , 1 , 1 , 1 , 1 , 1, 1, 1])
        crossLoss = nn.CrossEntropyLoss(weight= class_weight.cuda() ,reduction='mean', )
        self.ce = self.sigma *crossLoss(inputs, targets)

        inputs = torch.softmax(inputs, 1)

        # flatten label and prediction tensors
        inputs_f = inputs.view(-1)
        targets_f = targets.view(-1)
        inputs = inputs[:, 1:9, :, :].reshape(-1)
        targets = targets[:, 1:9, :, :].reshape(-1)

        # True Positives, False Positives & False Negatives
        TP = (inputs* targets).sum()
        FP = ((1 - targets_f) * inputs_f).sum()
        FN = (targets_f * (1 - inputs_f)).sum()

        self.tversky = self.theta * (1 - (TP + smooth) / (TP + self.alpha * FP + self.beta * FN + smooth))
        Dice_BCE = self.ce + self.tversky

        return Dice_BCE

Hyperparameters

In [ ]:
# Hyperparameters etc.
LEARNING_RATE = 1e-4
TRAIN_IMG_DIR = path+"trainset"
TEST_IMG_DIR = path+"testset"
BATCH_SIZE = 6
NUM_EPOCHS = 500

NUM_WORKERS = 2
IMAGE_HEIGHT = 256
IMAGE_WIDTH = 256
PIN_MEMORY = True
LOAD_MODEL = True
alpha = 0.3 #Tversky hyperparameters
beta = 0.7  #Tversky hyperparameters
sigma = 0.4 #Proportion of Cross entropy loss
theta = 0.6 #Proportion of Tversk loss

Train and check accuracy:

In [ ]:
def train_fn(loader, model, optimizer, loss_fn, scaler):
    loop = tqdm(loader)

    for batch_idx, all_data in enumerate(loop):
        data = all_data[:, 0, :, :]
        targets = all_data[:, 1:10, :, :]
        data = data.float().unsqueeze(1).to(device=DEVICE)
        targets = targets.float().to(device=DEVICE)

        # forward
        with torch.cuda.amp.autocast():
            predictions = model(data)
            loss = loss_fn(predictions, targets)

        # backward
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # update tqdm loop
        loop.set_postfix(loss=loss.item())

    print(f"Total Loss: {loss.item()}...Tversky loss: {loss_fn.tversky}... Cross Entropy: {loss_fn.ce}")

    # If model is on the top 5 save it
    checkpoint = {
        "state_dict": model.state_dict(),
        "optimizer": optimizer.state_dict(),
    }
    check_top_five(path+"/checkpoints/baseline/", "baseline.json", loss_fn.tversky.item(), checkpoint)

def main():
    train_transform = torchvision.transforms.Compose([
        Normalize(),
        Rescale(256),
        randomHorizontalFlip(),
        randomShiftScaleRotate(),
        randomHueSaturationValue(),
        randomZoom(),
        Grayscale(),
        ToTensor(),
    ])

    test_transforms  = torchvision.transforms.Compose([
        Normalize(),
        Rescale(256),
        Grayscale(),
        ToTensor(),
    ])

    model = UNET(in_channels=1, out_channels=9).to(DEVICE)
    loss_fn = Total_loss(alpha=alpha , beta=beta , sigma=sigma , theta=theta)
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE , weight_decay=1e-5 , amsgrad=True )
    scaler = torch.cuda.amp.GradScaler()

    train_loader, test_loader = get_loaders(
        TRAIN_IMG_DIR,
        TEST_IMG_DIR,
        BATCH_SIZE,
        train_transform,
        test_transforms,
        NUM_WORKERS,
        PIN_MEMORY,
    )

    if LOAD_MODEL:
        load_checkpoint(torch.load(path+"checkpoints/baseline/checkpoint_3.pth.tar"), model)

    for epoch in range(NUM_EPOCHS):
        print(f"Traing epoch: {epoch}.............................")
        train_fn(train_loader, model, optimizer, loss_fn, scaler)

        # check accuracy
        tmp_metrics = check_accuracy(train_loader ,test_loader, model, device=DEVICE)
        #Save metrics
        save_metrics(tmp_metrics,path+'/metrics/baseline.csv')

if __name__ == '__main__':
    main()


=> Loading checkpoint
Traing epoch: 0.............................


100%|██████████| 24/24 [00:30<00:00,  1.29s/it, loss=0.298]

Total Loss: 0.2980881929397583...Tversky loss: 0.24781741201877594... Cross Entropy: 0.05027078464627266
0.23324386775493622 > 0.24781741201877594????
Checkpoint is not in top 5 :'(


Testing set:...Recall: 86.51 ,...Car Recall: 70.99076843261719 ,...Dice score: 0.1349177360534668 ,...Tversky loss: 0.3484901487827301,...Cross_entropy: 0.13338243961334229
Training set:...Recall: 80.96 ,...Car Recall: 53.37736511230469 ,...Dice score: 0.21790534257888794 ,...Tversky loss: 0.4060618281364441,...Cross_entropy: 0.1991739720106125
Traing epoch: 1.............................


100%|██████████| 24/24 [00:30<00:00,  1.26s/it, loss=0.771]

...

Testing set:...Recall: 90.84 ,...Car Recall: 71.1034927368164 ,...Dice score: 0.0915873646736145 ,...Tversky loss: 0.2661850154399872,...Cross_entropy: 0.12057826668024063
Training set:...Recall: 88.90 ,...Car Recall: 68.77377319335938 ,...Dice score: 0.1574423909187317 ,...Tversky loss: 0.30431970953941345,...Cross_entropy: 0.12653879821300507
Traing epoch: 229.............................


100%|██████████| 24/24 [00:30<00:00,  1.28s/it, loss=0.243]

Total Loss: 0.2426181137561798...Tversky loss: 0.1650579571723938... Cross Entropy: 0.07756014913320541
0.13479502499103546 > 0.1650579571723938????
Checkpoint is not in top 5 :'(


Testing set:...Recall: 91.64 ,...Car Recall: 67.54569244384766 ,...Dice score: 0.08363848924636841 ,...Tversky loss: 0.2507879137992859,...Cross_entropy: 0.12482728809118271
Training set:...Recall: 87.27 ,...Car Recall: 58.41094207763672 ,...Dice score: 0.1671770215034485 ,...Tversky loss: 0.32328909635543823,...Cross_entropy: 0.20745807886123657
Traing epoch: 230.............................


 83%|████████▎ | 20/24 [00:26<00:05,  1.26s/it, loss=0.502]